In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

2023-04-27 13:22:45.641472: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
data = pd.read_csv('../data/haiku.csv')
data = data.replace("/", " / ", regex=True)
data = data.dropna()
data.head()

,Unnamed: 0,id,processed_title,ups,keywords
0,0,1020ac,There's nothing inside / There is nothing outs...,5,"[('inside', 0.5268), ('outside', 0.3751), ('se..."
1,1,107cob,From whole we crumble / Forever lost to chaos ...,1,"[('chaos', 0.5962), ('crumble', 0.4749), ('for..."
2,2,109a51,Indistinctiveness / Immeasurability / Capitalism,3,"[('indistinctiveness', 0.7664), ('immeasurabil..."
3,3,10eysi,Internet is down / Obligations go bye-bye / Of...,9,"[('office', 0.5033), ('obligations', 0.4663), ..."
4,4,10f79k,Cotton in my mouth / Needles in my blood and b...,1,"[('needles', 0.5314), ('cotton', 0.4806), ('bl..."


In [3]:
def tokenize(sentence: str):
    return list(sentence.lower().split())

def vectorize(tokens):
    vocab, index = {}, 1  # start indexing from 1
    vocab['<pad>'] = 0  # add a padding token
    for token in tokens:
        token = token.strip()
        if token not in vocab:
            vocab[token] = index
            index += 1
    return vocab

def find_max_length(vectorized_poems):
    max_length = 0
    for poem in vectorized_poems:
        # if len(poem)
        max_length = max(max_length, len(poem))   
    return max_length

In [4]:
all_text = " ".join(data["processed_title"].to_list())
tokens = tokenize(all_text)
vocab_map = vectorize(tokens)

In [5]:
data["vectorized"] = data["processed_title"].apply(lambda x: [vocab_map[t.strip()] for t in x.lower().split()])
data = data[[a.count(4) <= 2 for a in data['vectorized']]]
data = data[data['vectorized'].apply(lambda x: len(x) <= 19)]
data.head()

,Unnamed: 0,id,processed_title,ups,keywords,vectorized
0,0,1020ac,There's nothing inside / There is nothing outs...,5,"[('inside', 0.5268), ('outside', 0.3751), ('se...","[1, 2, 3, 4, 5, 6, 2, 7, 8, 4, 9, 10, 11, 12, 13]"
1,1,107cob,From whole we crumble / Forever lost to chaos ...,1,"[('chaos', 0.5962), ('crumble', 0.4749), ('for...","[14, 15, 16, 17, 4, 18, 19, 20, 21, 4, 22, 23,..."
2,2,109a51,Indistinctiveness / Immeasurability / Capitalism,3,"[('indistinctiveness', 0.7664), ('immeasurabil...","[25, 4, 26, 4, 27]"
3,3,10eysi,Internet is down / Obligations go bye-bye / Of...,9,"[('office', 0.5033), ('obligations', 0.4663), ...","[28, 6, 29, 4, 30, 31, 32, 4, 33, 34]"
4,4,10f79k,Cotton in my mouth / Needles in my blood and b...,1,"[('needles', 0.5314), ('cotton', 0.4806), ('bl...","[35, 12, 36, 37, 4, 38, 12, 36, 39, 40, 41, 4,..."


In [6]:
data_vectorized_list = data["vectorized"].to_list()

In [7]:
padded_data = pad_sequences(data_vectorized_list, padding='post')

In [8]:
haikus = np.array(padded_data)
haikus

array([[    1,     2,     3, ...,     0,     0,     0],
       [   14,    15,    16, ...,     0,     0,     0],
       [   25,     4,    26, ...,     0,     0,     0],
       ...,
       [  572,  6718, 24145, ...,     0,     0,     0],
       [ 1138,  7924,  1572, ...,     0,     0,     0],
       [  422, 24147,     4, ...,     0,     0,     0]], dtype=int32)

In [67]:
window_size = 6

In [68]:
training_examples = []

for haiku in haikus:
    for i in range(len(haiku) - window_size + 1):
        input_words = haiku[i:i+window_size-1]
        output_word = haiku[i+window_size-1]
        training_examples.append((input_words, output_word))
training_examples[:20]

[(array([1, 2, 3, 4, 5], dtype=int32), 6),
 (array([2, 3, 4, 5, 6], dtype=int32), 2),
 (array([3, 4, 5, 6, 2], dtype=int32), 7),
 (array([4, 5, 6, 2, 7], dtype=int32), 8),
 (array([5, 6, 2, 7, 8], dtype=int32), 4),
 (array([6, 2, 7, 8, 4], dtype=int32), 9),
 (array([2, 7, 8, 4, 9], dtype=int32), 10),
 (array([ 7,  8,  4,  9, 10], dtype=int32), 11),
 (array([ 8,  4,  9, 10, 11], dtype=int32), 12),
 (array([ 4,  9, 10, 11, 12], dtype=int32), 13),
 (array([ 9, 10, 11, 12, 13], dtype=int32), 0),
 (array([10, 11, 12, 13,  0], dtype=int32), 0),
 (array([11, 12, 13,  0,  0], dtype=int32), 0),
 (array([12, 13,  0,  0,  0], dtype=int32), 0),
 (array([14, 15, 16, 17,  4], dtype=int32), 18),
 (array([15, 16, 17,  4, 18], dtype=int32), 19),
 (array([16, 17,  4, 18, 19], dtype=int32), 20),
 (array([17,  4, 18, 19, 20], dtype=int32), 21),
 (array([ 4, 18, 19, 20, 21], dtype=int32), 4),
 (array([18, 19, 20, 21,  4], dtype=int32), 22)]

In [69]:
from keras.models import Sequential
from keras.layers import LSTM, Embedding, Dense, SimpleRNN
from sklearn.model_selection import train_test_split

In [70]:
vocab_map

{'<pad>': 0,
 "there's": 1,
 'nothing': 2,
 'inside': 3,
 '/': 4,
 'there': 5,
 'is': 6,
 'outside': 7,
 'me': 8,
 'i': 9,
 'search': 10,
 'on': 11,
 'in': 12,
 'hope.': 13,
 'from': 14,
 'whole': 15,
 'we': 16,
 'crumble': 17,
 'forever': 18,
 'lost': 19,
 'to': 20,
 'chaos': 21,
 'never': 22,
 'one': 23,
 'again': 24,
 'indistinctiveness': 25,
 'immeasurability': 26,
 'capitalism': 27,
 'internet': 28,
 'down': 29,
 'obligations': 30,
 'go': 31,
 'bye-bye': 32,
 'office': 33,
 'rejoices': 34,
 'cotton': 35,
 'my': 36,
 'mouth': 37,
 'needles': 38,
 'blood': 39,
 'and': 40,
 'bones': 41,
 'hammers': 42,
 'head.': 43,
 'mighty': 44,
 'hummingbird': 45,
 'drinks': 46,
 'a': 47,
 "grapefruit's": 48,
 'blossom;': 49,
 'blots': 50,
 'out': 51,
 'an': 52,
 'airplane': 53,
 'downvotes': 54,
 'fall': 55,
 'as': 56,
 'sharp': 57,
 'snowflakes': 58,
 'of': 59,
 'early': 60,
 'winter,': 61,
 'execution.': 62,
 'seven': 63,
 'ships': 64,
 'tonight': 65,
 'guess': 66,
 "should've": 67,
 'said': 68

In [71]:
import keras.backend as K

def masked_loss_function(y_true, y_pred):
    mask = K.cast(K.not_equal(y_true, 0), K.floatx())
    loss = K.sparse_categorical_crossentropy(y_true, y_pred)
    masked_loss = loss * mask
    return K.sum(masked_loss) / K.sum(mask)


In [72]:
vocab_size = len(vocab_map)
embedding_size = 64
input_length = window_size - 1
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_size, input_length=input_length),
    SimpleRNN(32),
    Dense(128, activation='relu'),
    Dense(vocab_size, activation='softmax')
])


In [73]:
model.compile(loss=masked_loss_function, optimizer='adam', metrics=['accuracy'])

In [86]:
x = np.array([example[0] for example in training_examples])
y = np.array([example[1] for example in training_examples])

In [87]:
pd.DataFrame(x).head(20)

,0,1,2,3,4
0,1,2,3,4,5
1,2,3,4,5,6
2,3,4,5,6,2
3,4,5,6,2,7
4,5,6,2,7,8
5,6,2,7,8,4
6,2,7,8,4,9
7,7,8,4,9,10
8,8,4,9,10,11
9,4,9,10,11,12


In [88]:
model.fit(x, y, batch_size=32, epochs=1)

6654/6654 [==============================] - 242s 36ms/step - loss: 154.8871 - accuracy: 0.3817


In [89]:
vocab_map_inv = dict([(value, key) for key, value in vocab_map.items()])
vocab_map_inv

{0: '<pad>',
 1: "there's",
 2: 'nothing',
 3: 'inside',
 4: '/',
 5: 'there',
 6: 'is',
 7: 'outside',
 8: 'me',
 9: 'i',
 10: 'search',
 11: 'on',
 12: 'in',
 13: 'hope.',
 14: 'from',
 15: 'whole',
 16: 'we',
 17: 'crumble',
 18: 'forever',
 19: 'lost',
 20: 'to',
 21: 'chaos',
 22: 'never',
 23: 'one',
 24: 'again',
 25: 'indistinctiveness',
 26: 'immeasurability',
 27: 'capitalism',
 28: 'internet',
 29: 'down',
 30: 'obligations',
 31: 'go',
 32: 'bye-bye',
 33: 'office',
 34: 'rejoices',
 35: 'cotton',
 36: 'my',
 37: 'mouth',
 38: 'needles',
 39: 'blood',
 40: 'and',
 41: 'bones',
 42: 'hammers',
 43: 'head.',
 44: 'mighty',
 45: 'hummingbird',
 46: 'drinks',
 47: 'a',
 48: "grapefruit's",
 49: 'blossom;',
 50: 'blots',
 51: 'out',
 52: 'an',
 53: 'airplane',
 54: 'downvotes',
 55: 'fall',
 56: 'as',
 57: 'sharp',
 58: 'snowflakes',
 59: 'of',
 60: 'early',
 61: 'winter,',
 62: 'execution.',
 63: 'seven',
 64: 'ships',
 65: 'tonight',
 66: 'guess',
 67: "should've",
 68: 'said'

In [106]:
def generate_poem(input_words):
    vectorized_input = [vocab_map[word] for word in input_words]
    print(f"User specified words {input_words} which were vectorized as {vectorized_input}")
    output_poem = input_words
    for i in range(19 - window_size - 1):
        input = np.array(vectorized_input[i:i+window_size -1]).reshape((1,window_size -1))
        prediction = np.array(model.predict(input, verbose=0))
        new_word_vector = (prediction[0].argsort()[::-1])[np.random.randint(0, 2)]
        vectorized_input.append(new_word_vector)
        new_word = vocab_map_inv[new_word_vector]
        output_poem.append(new_word)
    output = " ".join(output_poem)
    print(f"OUTPUT POEM: {output}")

        
                                   


In [107]:
generate_poem(["internet", "is", "down", "and", "i"])

User specified words ['internet', 'is', 'down', 'and', 'i'] which were vectorized as [28, 6, 29, 40, 9]
OUTPUT POEM: internet is down and i <pad> <pad> <pad> / the a / i am the / the


In [108]:
generate_poem(["fun", "today", "and", "tomorrow", "together"])

User specified words ['fun', 'today', 'and', 'tomorrow', 'together'] which were vectorized as [158, 648, 40, 1660, 2213]
OUTPUT POEM: fun today and tomorrow together <pad> <pad> / the a / the the / the a <pad>


In [109]:
generate_poem(["the", "bear", "and", "i", "cherry"])

User specified words ['the', 'bear', 'and', 'i', 'cherry'] which were vectorized as [85, 1014, 40, 9, 1264]
OUTPUT POEM: the bear and i cherry <pad> <pad> / the the / i i i i you <pad>


In [110]:
generate_poem(["i", "am", "midnight", "for", "tomorrow"])

User specified words ['i', 'am', 'midnight', 'for', 'tomorrow'] which were vectorized as [9, 155, 2577, 208, 1660]
OUTPUT POEM: i am midnight for tomorrow <pad> / the the <pad> to the <pad> / i am i
